In [2]:
import pandas as pd

In [35]:
#Global paramenters
#API URLs from RxNorm
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
#RxNorm methods
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
#dataframe
df = pd.DataFrame()

In [36]:
#Method to get drug with only the opioid ingredient
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df

In [37]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

In [ ]:
base_url_3 = "https://rxnav.nlm.nih.gov/REST/rxcui.json?name="


In [13]:
antidep_ingredients = pd.read_excel("Antidepressants_Ingredients.xlsx", header=None)

In [14]:
antidep_ingredients

,0
0,5-hydroxytryptophan
1,amineptin
2,amitriptyline
3,amoxapine
4,bifemelane
5,brexanolone
6,bupropion
7,butriptyline
8,citalopram
9,clomipramine


All Antidepressants Ingredients

In [30]:
drugs = []
for drug in antidep_ingredients[0]:
    ingredient = pd.read_json("https://rxnav.nlm.nih.gov/REST/rxcui.json?name="+drug)
    drugs.append(ingredient.idGroup.rxnormId[0])


In [33]:
len(drugs)

57

In [38]:
#loop array to get complete list of drugs
df_antidep = pd.DataFrame()
for drug in drugs:
    self_df = query_self(drug)
    merg_df = query_drug(drug)
    df_antidep = pd.concat([df_antidep, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_antidep = pd.concat([df_antidep, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_antidep

,rxcui,name,synonym,tty,language,suppress,umlscui
0,94,5-hydroxytryptophan,oxitriptan,IN,ENG,N,
1,1293415,Somnicin,,BN,ENG,N,
2,1008781,5-hydroxytryptophan / vitamin B6,,MIN,ENG,N,
3,1293412,5-hydroxytryptophan / magnesium oxide / melato...,,MIN,ENG,N,
4,1945211,5-hydroxytryptophan / melatonin,,MIN,ENG,N,
...,...,...,...,...,...,...,...
1611,1439811,vortioxetine 5 MG,,SCDC,ENG,N,
1612,1439839,vortioxetine 15 MG,,SCDC,ENG,N,
1613,1439807,vortioxetine Oral Tablet,,SCDF,ENG,N,
1614,1439805,vortioxetine Oral Product,,SCDG,ENG,N,


In [39]:
#Export
df_antidep.to_excel("New_Complete_Antidepressants_List.xlsx", index=False)